In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1RJjttCvltRK-j5XaI_Tp752cibGKRYMf", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/02_00_intro.mp3"))


In [ ]:
#@title 🎧 Code Walkthrough: Setup Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_01_setup_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 What to Look For: 1d Gaussian Score Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_06_1d_gaussian_score_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Why Matter
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_02_why_matter.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Compass Analogy
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_03_compass_analogy.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Math Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_04_math_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Key Insight Ebm
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_05_key_insight_ebm.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


# The Score Function and Langevin Dynamics -- Vizuara

## 1. Why Does This Matter?

In the previous notebook, we saw that the Boltzmann distribution converts energy to probability, but the partition function $Z$ is intractable. This means we cannot directly train energy-based models or sample from them using the probability density.

The **score function** solves both problems elegantly. It is the gradient of the log probability density, and it completely bypasses the partition function. Combined with **Langevin dynamics**, it gives us a way to generate samples from any distribution -- even one we cannot normalize.

**By the end of this notebook, you will:**
- Compute score functions analytically for simple distributions
- Implement Langevin dynamics to sample from known distributions
- Visualize score vector fields in 2D
- Understand why the score function eliminates the partition function

## 2. Building Intuition

### The Compass Analogy

Imagine you are lost in a thick fog on a vast landscape. You cannot see the terrain, but you have a special compass. Instead of pointing north, this compass always points toward the nearest valley -- the region where the terrain is lowest (and where data is most likely).

This compass is the **score function**. At every point in space, it tells you: "Go this way to reach higher probability."

### Think About This

Before we write any equations:
- If you have a Gaussian distribution centered at 0, which direction should the score point at $x = 3$? At $x = -2$? At $x = 0$?
- Why might we prefer to model gradients (directions) rather than densities (heights)?

## 3. The Mathematics

The score function is defined as:

$$s(x) = \nabla_x \log p(x)$$

This is a **vector field** -- at every point $x$, it gives a vector pointing in the direction of increasing log probability.

For a 1D standard Gaussian $p(x) = \frac{1}{\sqrt{2\pi}} \exp(-x^2/2)$:

$$\log p(x) = -\frac{1}{2}\log(2\pi) - \frac{x^2}{2}$$

$$s(x) = \frac{d}{dx}\log p(x) = -x$$

Computationally, this means: the score at any point $x$ is simply $-x$. At $x=3$, the score is $-3$ (move left). At $x=-2$, the score is $2$ (move right). At $x=0$, the score is $0$ (you are at the peak -- stay put).

**The key insight for EBMs:**

$$s(x) = \nabla_x \log \frac{\exp(-E(x))}{Z} = -\nabla_x E(x) - \nabla_x \log Z = -\nabla_x E(x)$$

The $\log Z$ term vanishes because $Z$ does not depend on $x$.

## 4. Let's Build It -- Component by Component

### 4.1 Computing Scores Analytically

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

def gaussian_score(x, mean=0.0, std=1.0):
    """
    Analytical score function for a 1D Gaussian.
    s(x) = -(x - mean) / std^2
    """
    return -(x - mean) / (std ** 2)

# Verify: score should point toward the mean
x = torch.linspace(-4, 4, 500)
scores = gaussian_score(x, mean=0.0, std=1.0)

fig, axes = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

# Top: probability density
p = torch.exp(-x**2 / 2) / (2 * np.pi) ** 0.5
axes[0].plot(x.numpy(), p.numpy(), 'b-', linewidth=2)
axes[0].fill_between(x.numpy(), p.numpy(), alpha=0.1, color='blue')
axes[0].set_ylabel('p(x)', fontsize=12)
axes[0].set_title('Gaussian Probability Density', fontsize=14)
axes[0].grid(True, alpha=0.3)

# Bottom: score function
axes[1].plot(x.numpy(), scores.numpy(), 'r-', linewidth=2)
axes[1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[1].set_ylabel('s(x) = -x', fontsize=12)
axes[1].set_xlabel('x', fontsize=12)
axes[1].set_title('Score Function', fontsize=14)
axes[1].grid(True, alpha=0.3)

# Annotate key points
for xi, color in [(2.0, 'green'), (-3.0, 'orange'), (0.0, 'purple')]:
    si = gaussian_score(torch.tensor(xi)).item()
    axes[1].annotate(f's({xi:.0f}) = {si:.0f}',
        xy=(xi, si), fontsize=11, fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.tight_layout()
plt.show()
print("The score always points toward x=0 (the peak). This is exactly what we want.")

In [ ]:
#@title 🎧 Transition: 2d Score Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_07_2d_score_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 Score Fields in 2D

In [ ]:
#@title 🎧 What to Look For: 2d Gaussian Score Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_08_2d_gaussian_score_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def gaussian_2d_score(x, mean=None, cov_inv=None):
    """
    Score function for a 2D Gaussian.
    s(x) = -Sigma_inv @ (x - mean)
    """
    if mean is None:
        mean = torch.zeros(2)
    if cov_inv is None:
        cov_inv = torch.eye(2)
    return -torch.matmul(x - mean, cov_inv.T)

# Create a grid and compute score field
n_grid = 15
x1 = torch.linspace(-3, 3, n_grid)
x2 = torch.linspace(-3, 3, n_grid)
X1, X2 = torch.meshgrid(x1, x2, indexing='ij')
grid_points = torch.stack([X1.flatten(), X2.flatten()], dim=-1)

# Compute scores
scores_2d = gaussian_2d_score(grid_points)
S1 = scores_2d[:, 0].reshape(n_grid, n_grid)
S2 = scores_2d[:, 1].reshape(n_grid, n_grid)

# Also compute the density for the background
x_fine = torch.linspace(-3, 3, 100)
X1f, X2f = torch.meshgrid(x_fine, x_fine, indexing='ij')
density = torch.exp(-0.5 * (X1f**2 + X2f**2))

fig, ax = plt.subplots(figsize=(8, 8))
ax.contourf(X1f.numpy(), X2f.numpy(), density.numpy(), levels=20, cmap='Blues', alpha=0.5)
ax.quiver(X1.numpy(), X2.numpy(), S1.numpy(), S2.numpy(),
          color='darkred', scale=40, width=0.004)
ax.set_xlabel('x1', fontsize=12)
ax.set_ylabel('x2', fontsize=12)
ax.set_title('Score Field of a 2D Gaussian', fontsize=14)
ax.set_aspect('equal')
plt.tight_layout()
plt.show()
print("All arrows point toward the center -- the score field guides you to high density.")

In [ ]:
#@title 🎧 Transition: Score From Energy Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_09_score_from_energy_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 Score from Energy Functions

In [ ]:
#@title 🎧 What to Look For: Score From Energy Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_10_score_from_energy_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def score_from_energy(energy_fn, x):
    """
    Compute the score function from an energy function using autograd.
    s(x) = -grad_x E(x)

    This is the KEY insight: we never need to compute Z!
    """
    x = x.clone().requires_grad_(True)
    E = energy_fn(x).sum()
    grad_E = torch.autograd.grad(E, x)[0]
    return -grad_E

# Example: double-well energy
def double_well(x):
    return (x[:, 0]**2 + x[:, 1]**2 - 1)**2 + 0.5 * x[:, 1]**2

# Compute score field
grid_points_2d = torch.stack([X1.flatten(), X2.flatten()], dim=-1)
scores_dw = score_from_energy(double_well, grid_points_2d)
S1_dw = scores_dw[:, 0].reshape(n_grid, n_grid).detach()
S2_dw = scores_dw[:, 1].reshape(n_grid, n_grid).detach()

# Compute energy for background
E_bg = double_well(torch.stack([X1f.flatten(), X2f.flatten()], dim=-1))
E_bg = E_bg.reshape(100, 100).detach()

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].contourf(X1f.numpy(), X2f.numpy(), E_bg.numpy(), levels=30, cmap='viridis')
axes[0].set_title('Energy Landscape', fontsize=14)
axes[0].set_xlabel('x1')
axes[0].set_ylabel('x2')

axes[1].contourf(X1f.numpy(), X2f.numpy(), torch.exp(-E_bg).numpy(), levels=30, cmap='hot')
axes[1].quiver(X1.numpy(), X2.numpy(), S1_dw.numpy(), S2_dw.numpy(),
               color='white', scale=50, width=0.004)
axes[1].set_title('Probability + Score Field', fontsize=14)
axes[1].set_xlabel('x1')
axes[1].set_ylabel('x2')

for ax in axes:
    ax.set_aspect('equal')

plt.tight_layout()
plt.show()
print("Score field points toward low-energy (high-probability) regions -- no Z needed!")

In [ ]:
#@title 🎧 Before You Start: Todo Langevin Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_11_todo_langevin_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. Your Turn -- Implement Langevin Dynamics

### The Langevin Update Rule

Langevin dynamics generates samples by iteratively following the score with added noise:

$$x_{t+1} = x_t + \eta \cdot s(x_t) + \sqrt{2\eta} \cdot \epsilon, \quad \epsilon \sim \mathcal{N}(0, I)$$

In [ ]:
#@title 🎧 Before You Start: Todo Langevin 1d
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_12_todo_langevin_1d.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def langevin_dynamics(score_fn, x_init, n_steps=1000, step_size=0.01):
    """
    TODO: Implement Langevin dynamics sampling.

    Args:
        score_fn: Function that takes x and returns the score s(x)
        x_init: Initial point (tensor)
        n_steps: Number of Langevin steps
        step_size: Step size (eta)

    Returns:
        final_x: The final sample
        trajectory: List of all intermediate points

    The update rule is:
        x_{t+1} = x_t + eta * score(x_t) + sqrt(2 * eta) * noise

    Hint: noise = torch.randn_like(x)
    """
    x = x_init.clone()
    trajectory = [x.clone()]

    for t in range(n_steps):
        # ============ TODO ============
        # Step 1: Compute the score at current x
        # Step 2: Sample Gaussian noise
        # Step 3: Update x using the Langevin rule
        # ==============================

        score = ???  # YOUR CODE HERE
        noise = ???  # YOUR CODE HERE
        x = ???      # YOUR CODE HERE

        trajectory.append(x.clone())

    return x, trajectory

# Uncomment to test after completing TODO:
# score_fn = lambda x: -x  # Score for standard Gaussian
# x_init = torch.tensor([4.0])
# final, traj = langevin_dynamics(score_fn, x_init, n_steps=500, step_size=0.01)
# print(f"Started at {x_init.item():.2f}, ended at {final.item():.2f}")

In [ ]:
#@title 🎧 Code Walkthrough: Verification Langevin 1d
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_13_verification_langevin_1d.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Verification cell
def langevin_dynamics_solution(score_fn, x_init, n_steps=1000, step_size=0.01):
    x = x_init.clone()
    trajectory = [x.clone()]
    for t in range(n_steps):
        score = score_fn(x)
        noise = torch.randn_like(x)
        x = x + step_size * score + (2 * step_size) ** 0.5 * noise
        trajectory.append(x.clone())
    return x, trajectory

score_fn = lambda x: -x
x_init = torch.tensor([4.0])
final, traj = langevin_dynamics_solution(score_fn, x_init, n_steps=1000, step_size=0.01)

# The final sample should be close to 0 (the mean of the standard Gaussian)
assert abs(final.item()) < 2.0, f"Expected final sample near 0, got {final.item():.2f}"
print(f"Started at {x_init.item():.2f}, ended at {final.item():.2f}")
print("Correct! The sample moved toward the peak of the Gaussian.")

In [ ]:
#@title 🎧 Before You Start: Todo Langevin 2d Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_14_todo_langevin_2d_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5.2 Your Turn -- 2D Langevin Sampling

In [ ]:
#@title 🎧 Before You Start: Todo Langevin 2d
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_15_todo_langevin_2d.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def langevin_2d(score_fn, n_samples=200, n_steps=500, step_size=0.01):
    """
    TODO: Run Langevin dynamics for multiple 2D samples simultaneously.

    Start each sample from a random point (e.g., torch.randn(n_samples, 2) * 3)
    and run n_steps of Langevin dynamics.

    Return the final samples and a few trajectories for visualization.
    """
    x = torch.randn(n_samples, 2) * 3  # Random initial points
    trajectories = [x[:5].clone()]  # Track first 5 trajectories

    for t in range(n_steps):
        # ============ TODO ============
        # Same as 1D but for 2D vectors
        # ==============================

        score = ???  # YOUR CODE HERE
        noise = ???  # YOUR CODE HERE
        x = ???      # YOUR CODE HERE
        trajectories.append(x[:5].clone())

    return x, trajectories

In [ ]:
#@title 🎧 Transition: Putting It Together Transition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_16_putting_it_together_transition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together

In [ ]:
#@title 🎧 What to Look For: 2d Langevin Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_17_2d_langevin_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Complete Langevin sampling from a 2D Gaussian
score_fn_2d = lambda x: -x  # Score for standard 2D Gaussian

x = torch.randn(500, 2) * 4  # Start far from center
trajectories = [x[:10].clone()]

for t in range(1000):
    score = score_fn_2d(x)
    noise = torch.randn_like(x)
    x = x + 0.01 * score + (0.02) ** 0.5 * noise
    if t % 50 == 0:
        trajectories.append(x[:10].clone())

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Starting points
axes[0].scatter(trajectories[0][:, 0], trajectories[0][:, 1], c='red', s=20, alpha=0.7)
axes[0].set_title('Initial Points', fontsize=14)
axes[0].set_xlim(-5, 5)
axes[0].set_ylim(-5, 5)
axes[0].set_aspect('equal')

# Trajectories
for i in range(min(5, len(trajectories[0]))):
    traj_x = [t[i, 0].item() for t in trajectories]
    traj_y = [t[i, 1].item() for t in trajectories]
    axes[1].plot(traj_x, traj_y, '-', alpha=0.5, linewidth=1)
    axes[1].plot(traj_x[0], traj_y[0], 'go', markersize=6)
    axes[1].plot(traj_x[-1], traj_y[-1], 'bs', markersize=6)
axes[1].set_title('Langevin Trajectories', fontsize=14)
axes[1].set_xlim(-5, 5)
axes[1].set_ylim(-5, 5)
axes[1].set_aspect('equal')

# Final samples
axes[2].scatter(x[:, 0].numpy(), x[:, 1].numpy(), c='blue', s=5, alpha=0.3)
theta = torch.linspace(0, 2*np.pi, 100)
for r in [1, 2]:
    axes[2].plot(r*torch.cos(theta).numpy(), r*torch.sin(theta).numpy(),
                 'r--', alpha=0.3)
axes[2].set_title('Final Samples', fontsize=14)
axes[2].set_xlim(-5, 5)
axes[2].set_ylim(-5, 5)
axes[2].set_aspect('equal')

plt.suptitle('Langevin Dynamics: Sampling from a 2D Gaussian', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()
print("The samples converge to the target Gaussian distribution!")

In [ ]:
#@title 🎧 Transition: Bimodal Sampling Transition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_18_bimodal_sampling_transition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. Training and Results -- Bimodal Sampling

In [ ]:
#@title 🎧 What to Look For: Bimodal Sampling Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_19_bimodal_sampling_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Challenge: sample from a bimodal distribution using Langevin dynamics

def bimodal_score(x):
    """Score function for a mixture of two 2D Gaussians."""
    mu1 = torch.tensor([-2.0, 0.0])
    mu2 = torch.tensor([2.0, 0.0])

    # Unnormalized density contributions
    p1 = torch.exp(-0.5 * ((x - mu1) ** 2).sum(dim=-1, keepdim=True))
    p2 = torch.exp(-0.5 * ((x - mu2) ** 2).sum(dim=-1, keepdim=True))
    p_total = p1 + p2

    # Score = weighted sum of individual Gaussian scores
    s1 = -(x - mu1)
    s2 = -(x - mu2)
    score = (p1 * s1 + p2 * s2) / p_total
    return score

# Run Langevin dynamics
x = torch.randn(1000, 2) * 4
for t in range(2000):
    score = bimodal_score(x)
    noise = torch.randn_like(x)
    x = x + 0.005 * score + (0.01) ** 0.5 * noise

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Score field
n_g = 20
g1 = torch.linspace(-5, 5, n_g)
g2 = torch.linspace(-4, 4, n_g)
G1, G2 = torch.meshgrid(g1, g2, indexing='ij')
gp = torch.stack([G1.flatten(), G2.flatten()], dim=-1)
gs = bimodal_score(gp)
axes[0].quiver(G1.numpy(), G2.numpy(),
               gs[:, 0].reshape(n_g, n_g).numpy(),
               gs[:, 1].reshape(n_g, n_g).numpy(),
               color='darkblue', scale=30)
axes[0].set_title('Score Field (Bimodal)', fontsize=14)
axes[0].set_aspect('equal')

# Generated samples
axes[1].scatter(x[:, 0].numpy(), x[:, 1].numpy(), c='coral', s=5, alpha=0.5)
axes[1].set_title('Langevin Samples', fontsize=14)
axes[1].set_aspect('equal')

plt.suptitle('Sampling from a Bimodal Distribution', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()
print("The samples form two clusters -- matching the bimodal target!")

In [ ]:
#@title 🎧 Transition: Final Output Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_20_final_output_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Final Output

In [ ]:
#@title 🎧 What to Look For: Animated Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_21_animated_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Animated Langevin dynamics visualization
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

x_anim = torch.randn(300, 2) * 4
frames_data = [x_anim.clone().numpy()]

for t in range(200):
    score = bimodal_score(x_anim)
    noise = torch.randn_like(x_anim)
    x_anim = x_anim + 0.01 * score + (0.02) ** 0.5 * noise
    if t % 2 == 0:
        frames_data.append(x_anim.clone().numpy())

fig, ax = plt.subplots(figsize=(8, 6))
scatter = ax.scatter([], [], c='steelblue', s=10, alpha=0.5)
ax.set_xlim(-6, 6)
ax.set_ylim(-5, 5)
ax.set_aspect('equal')

def update(frame):
    scatter.set_offsets(frames_data[frame])
    ax.set_title(f'Langevin Dynamics (Step {frame * 2})', fontsize=14)
    return scatter,

anim = FuncAnimation(fig, update, frames=len(frames_data), interval=50, blit=True)
plt.close()
HTML(anim.to_html5_video())

In [ ]:
#@title 🎧 Narration: Reflection
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_22_reflection.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Reflection and Next Steps

### Think About These Questions:
1. What happens if you set the noise to zero in Langevin dynamics? (Hint: it becomes gradient descent -- what problem does that cause?)
2. Why does the score function bypass the partition function? Can you explain this intuitively?
3. In this notebook, we used the TRUE score function. But for real data, we do not know the true distribution. How can we LEARN the score from data samples?

### What's Next
In the next notebook, we will learn **score matching** -- a technique to train a neural network to approximate the score function using only data samples, without ever knowing the true distribution.